In [3]:
def order_crossover(parent1, parent2):
    """
    Order Crossover (OX) for TSP.
    """
    # Ensure the first parent is the shorter one (or equal)
    if len(parent1) > len(parent2):
        parent1, parent2 = parent2, parent1

    # Randomly select a subset of parent1
    start_pos = random.randint(0, len(parent1) - 2)
    end_pos = random.randint(start_pos + 1, len(parent1) - 1)

    child1 = [None] * len(parent1)
    child2 = [None] * len(parent2)

    # Copy the subset to child1 and child2
    child1[start_pos:end_pos] = parent1[start_pos:end_pos]
    child2[start_pos:end_pos] = parent2[start_pos:end_pos]

    # For the rest positions in the child
    # If the position is occupied, insert the first unoccupied element from the other parent
    for i in range(len(parent1)):
        if not child1[i]:
            for gene in parent2:
                if gene not in child1:
                    child1[i] = gene
                    break
        if not child2[i]:
            for gene in parent1:
                if gene not in child2:
                    child2[i] = gene
                    break

    return child1, child2

def crossover(parents, crossover_rate, max_attempts=10000):
    """
    Modified crossover function using Order Crossover (OX) with validity check
    and multiple attempts to generate a valid child.
    """
    if random.random() >= crossover_rate:
        return parents[0][:]
    
    for _ in range(max_attempts):
        child1, child2 = order_crossover(parents[0], parents[1])
        
        if is_valid_individual(child1):
            # Return the child with the better score
            return child1 if get_score(child1) > get_score(child2) else child2
        
    # If after max_attempts we still don't have a valid child, return one of the parents
    return parents[0]


def find_target(nested_list, target):
    # 将二级列表转换为NumPy数组
    nested_array = np.array(nested_list)
    
    # 获取所有子列表的第一个元素
    first_elements = nested_array[:, 0]
    
    # 使用NumPy的索引功能来查找目标数字
    indices = np.where(first_elements == target)[0]
    
    if len(indices) == 0:
        print("Target not found! Nested List:", nested_list, "Target:", target)
    
    return indices

def calculate_enter_time_individual(path):
    individual = path.copy()
    for i in range(len(individual)):
        individual[i] = [[0, 0, 0]] + individual[i]
    timetable = {i: 0 for i in range(1, 8)}
    timetable[7] = float('inf')

    n = len(individual)
    result = [[0] for _ in range(n)]
    variables = {f"tourist_{i + 1}": 1 for i in range(n)}
    while(variables['tourist_1'] <=7 or variables['tourist_2'] <=7 or variables['tourist_3'] <=7):
        time = float("inf")
        ind = 0
        for i in range(len(result)):
            site = variables[f"tourist_{i+1}"]
            if site ==8:
                continue
            current_node = individual[i][site][0]
            previous_node = individual[i][site - 1][0]
            arrive_time = result[i][-1] + adj_matrix[previous_node][current_node]/individual[i][site][2] + individual[i][site-1][1]
            if arrive_time < time:
                ind = i
                time = arrive_time
        site = variables[f"tourist_{ind+1}"]
        arrive_time = time
        index = individual[ind][site][0]
        if index == 3:
            if arrive_time < timetable[index] - 1e-9:
                arrive_time = timetable[index]
                timetable[index] = arrive_time + individual[ind][site][1]
            elif abs(round(arrive_time / 30) * 30 - arrive_time)> 1e-9 and arrive_time > timetable[index] + 1e-9:
                    num = copy.deepcopy(arrive_time)
                    integer_part = int(num)
                    decimal_part = num - integer_part
                    remainder = integer_part % 30
                    difference = 30 - remainder - decimal_part

                    timetable[index] = individual[ind][site][1] + difference + arrive_time
                    arrive_time = difference + arrive_time                                    
            else:
                timetable[index] = timetable[index] +30
        elif index == 7:
            arrive_time = arrive_time
        else:
            if arrive_time > timetable[index]:
                 timetable[index] = arrive_time + individual[ind][site][1]
            else:
                 arrive_time = timetable[index]
                 timetable[index] = timetable[index] + individual[ind][site][1]               
        result[ind].append(arrive_time)
        variables[f"tourist_{ind+1}"] = variables[f"tourist_{ind+1}"] +1       
    return result

def calculate_enter_time_tourist(path):
    result = [0]
    for site in range(1, len(path)):
        current_node = path[site][0]
        previous_node = path[site - 1][0]
        arrive_time = result[-1] + adj_matrix[previous_node][current_node]/path[site][2] + path[site-1][1]
        result.append(arrive_time)
    return result


def is_valid_individual(individual):
    """
    判断个体是否满足约束条件
    """
    enter_time = calculate_enter_time_individual(individual)
    if any(et[-1] > 300 for et in enter_time):
        return False
    if any(et[find_target(ind, 1)[0]] > 240 for ind, et in zip(individual, enter_time)):
        return False
    return True

def is_valid_tourist(tourist):
    """
    判断个体是否满足约束条件
    """
    enter_time = calculate_enter_time_tourist(tourist)
    if enter_time[-1] > 300:
            return False
    if enter_time[find_target(tourist,1)[0]] > 240:
        return False
    return True


def generate_speeds(min_speed,max_speed,average_speed,num,iterations = 10000):
    #使用蒙特卡洛方法生成
    best_numbers = None
    best_average = 0
    
    for _ in range(iterations):
        # 生成一个包含 num 个在 min_speed 到 max_speed 之间的随机浮点数的列表
        numbers = [random.uniform(min_speed, max_speed) for _ in range(num)]
        
        # 计算生成数字的平均值
        average = sum(numbers) / len(numbers)
        
        # 检查平均值是否在所需范围内
        if average <= average_speed and average > best_average:
            best_numbers = numbers
            best_average = average
    best_numbers = [speed*1000/60 for speed in best_numbers]
    if best_numbers:
        return(best_numbers)
    else:
        print("在约束条件内未找到有效的数字组合。")

def mutate(individual, mutation_rate, max_attempts=10000):
    mutated_individual = copy.deepcopy(individual)
    global range_dict
    for _ in range(max_attempts):
        for ind in range(len(individual)):
            for i in range(0, len(mutated_individual[ind])-1):
                if random.random() < mutation_rate:
                    dimension = mutated_individual[ind][i][0]
                    if isinstance(range_dict[dimension], int):
                        mutated_individual[ind][i][1] = range_dict[dimension]
                    else:
                        mutated_individual[ind][i][1] = random.randint(range_dict[dimension][0], range_dict[dimension][1])
            if random.random() < mutation_rate:
                index1 = random.randint(0, len(mutated_individual[ind]) - 2)
                index2 = random.randint(0, len(mutated_individual[ind]) - 2)
                while index2 == index1:
                    index2 = random.randint(0, len(mutated_individual[ind]) - 2)
                    mutated_individual[ind][index1], mutated_individual[ind][index2] = mutated_individual[ind][index2], mutated_individual[ind][index1]
        if is_valid_individual(mutated_individual):
            break
    else:
        # 如果达到了最大尝试次数，可以返回原始的个体或采取其他措施
        mutated_individual = individual
    return mutated_individual

def get_score(individual):
    """
    评价一个个体
    """
    sightsee = 0
    entertime = calculate_enter_time_individual(individual)
    for i in range(len(entertime)):
        sightsee = sightsee + (330 - entertime[i][-1])
    for ind in range(len(individual)):
        for i in range(len(individual[ind])-1):
            sightsee = sightsee + individual[ind][i][1]
    return sightsee

def generate_individual(tourist_size):
    global range_dict
    keys = list(range_dict.keys())
    individual = []
    while len(individual) < tourist_size:
        path = [
            [dimension, range_dict[dimension] if isinstance(range_dict[dimension], int) else
             random.randint(range_dict[dimension][0], range_dict[dimension][1])]
            for dimension in keys[:6]
        ]
        
        # 随机打乱 path 中的元素顺序
        random.shuffle(path)
        path.append([7, float('inf')])
        # 添加速度信息
        speeds = [2000/60, 2000/60, 2000/60, 2000/60, 2000/60, 2000/60, 2000/60]
        for i in range(len(path)):
            path[i].append(speeds[i])
        if is_valid_tourist(path):
            individual.append(path)
    return individual

def initialize_population(population_size,tourist_size):
    """
    初始化种群，随机生成不同的路径作为初始解
    :param population_size: 种群大小
    :return: 生成的种群
    """
    # 构建一个字典表示对应的游览时间
    
    population = []
    while len(population) < population_size:
        individual = generate_individual(tourist_size)
        if is_valid_individual(individual):
            population.append(individual)
    return population

def elite_preserve(population, elite_size):
    """
    精英保留函数，从种群中选择最优个体作为精英个体，将精英个体直接传递到下一代
    :param population: 种群
    :param elite_size: 精英个体数量
    :return: 精英个体列表
    """
    elites = sorted(population, key=lambda path: get_score(path), reverse=True)[:elite_size]
    return elites

def select_parents(population):
    """
    从种群中选择两个父代
    """ 
    parents = random.sample(population, 2)
    parents_copy = copy.deepcopy(parents)  # 创建父代的深拷贝
    return parents_copy

def tournament_selection(population, tournament_size):
    """
    锦标赛选择，从种群中选择父代个体
    :param population: 种群
    :param tournament_size: 锦标赛大小
    :return: 选择出的父代个体
    """
    competitors = random.sample(population, tournament_size)
    winner = max(competitors, key=lambda individual: get_score(individual))
    return winner

def factor_probability4(population,parents, Pmin, Pmax, A = 9.903438):
    scores = [get_score(path) for path in population]  # 计算所有路径的分数
    fmean = np.mean(scores)
    f_ = max(get_score(parents[0]),get_score(parents[1]))
    if f_ <= fmean:
        return (Pmax - Pmin)/(1 + np.exp(A * 2 * (fmean - f_) / (fmean - Pmin -1))) + Pmin
    else:
        return Pmax

# 获取一个随机邻居的函数，通过在列表中更改一个值
def get_valid_neighbor(a):
    # 深度复制列表a，以确保原始列表不受更改的影响
    new_a = copy.deepcopy(a)
    while True:
        # 随机选择一个子列表
        sublist_idx = np.random.randint(len(new_a))
        # 从子列表中随机选择一个元素，该元素的索引不为7
        element_idx = np.random.choice([i for i, x in enumerate(new_a[sublist_idx]) if x[0] != 7])
        # 在其范围内更改其第二个值
        idx_val = new_a[sublist_idx][element_idx][0]
        range_val = range_dict[idx_val]
        # 检查范围值是否是元组，并随机选择该范围内的值
        if isinstance(range_val, tuple):
            new_a[sublist_idx][element_idx][1] = np.random.randint(range_val[0], range_val[1]+1)
        else:
            new_a[sublist_idx][element_idx][1] = range_val
        # 检查新的个体是否有效
        if is_valid_individual(new_a):
            break
    return new_a


# 使用模拟退火算法来最大化get_score(a)的函数
def simulated_annealing_maximize_v2(initial_temperature, alpha, num_iterations, a):
    # 复制a为当前解
    current_a = a.copy()
    # 计算当前解的得分
    current_score = get_score(current_a)
    # 初始化最佳解和得分
    best_a = current_a.copy()
    best_score = current_score

    # 设置初始温度
    T = initial_temperature
    # 开始模拟退火迭代
    for _ in range(num_iterations):
        # 获取当前解的一个邻居
        neighbor_a = get_valid_neighbor(current_a)
        # 计算邻居的得分
        neighbor_score = get_score(neighbor_a)

        # 计算能量差
        delta_E = neighbor_score - current_score
        # 如果新得分更好，或者满足一定条件，接受新解
        if delta_E > 0 or np.random.rand() < math.exp(delta_E / T):
            current_a = neighbor_a
            current_score = neighbor_score
            # 更新最佳解和得分
            if current_score > best_score:
                best_score = current_score
                best_a = current_a.copy()

        # 降低温度
        T *= alpha
    return best_a, best_score

import random
import copy
import numpy as np
import matplotlib.pyplot as plt
import math
def main():
    tourist_size = 3 #旅行团数量

    population_size = 50 # 种群大小
    num_generations = 1000 # 迭代次数
    tournament_size = 5 #锦标赛大小
    elite_size = 2 # 精英保留个体数量
    mutation_size = population_size - elite_size - tournament_size
    temperature_rate = 0.1
    initial_temperature = 100  # 初始温度
    alpha = 0.995              # 温度降低率
    num_iterations = 2000      # 迭代次数

    global range_dict
    range_dict = {
        1: (10, 30),
        2: (20, 60),
        3: 30,
        4: (30, 60),
        5: (20, 60),
        6: (30, 60),
    }

    global adj_matrix
    adj_matrix = (
    (  0, 300, 360, 210, 530, 475, 500, 690),
    (300,   0, 380, 270, 230, 285, 200, 390),
    (360, 380,   0, 510, 230, 665, 580, 770),
    (210, 270, 510,   0, 470, 265, 450, 640),
    (530, 230, 230, 470,   0, 515, 360, 550),
    (475, 285, 665, 265, 515,   0, 460, 650),
    (500, 200, 580, 450, 360, 460,   0, 190),
    (690, 390, 770, 640, 550, 650, 190,   0))
   # 初始化种群
    population = initialize_population(population_size,tourist_size)   
    # 记录每代的最佳时间
    best_times = []
        # 迭代指定的代数
    for generation in range(num_generations):
        new_population = []
        elites = elite_preserve(population, elite_size)       
        new_population.extend(elites)  # 将精英个体添加到新种群
        for _i1 in range(mutation_size):
            # 选择父代
            parents = select_parents(population)
            # 通过交叉产生子代
            crossover_rate = 1 - factor_probability4(population,parents, Pmin = 0.3, Pmax = 0.75, A = 9.903438)
            child = crossover(parents,crossover_rate)
            # 对子代进行变异
            mutation_rate = 1 - factor_probability4(population,parents, Pmin = 0.01, Pmax = 0.075, A = 9.903438)
            child = mutate(child, mutation_rate)
            # 将子代添加到新种群
            if random.random() < temperature_rate:
                child, _ = simulated_annealing_maximize_v2(initial_temperature, alpha, num_iterations, child)
            new_population.append(child)
            
        for _i2 in range(tournament_size):
            child = tournament_selection(population, tournament_size)
            new_population.append(child) 
        # 找到当前代的最佳路径和
        current_best_path = max(population, key=lambda path: get_score(path))
        current_best_distance = get_score(current_best_path)
        # 记录最佳时间
        best_times.append(current_best_distance)

        # 打印当前代的最佳路径和距离
        print(f"Generation {generation + 1}:")
        print("最佳路径：", current_best_path)
        print("最佳时间：", current_best_distance)
        print()
        # 更新种群
        population = new_population

    # 绘制迭代图
    plt.plot(range(1, num_generations + 1), best_times)
    plt.xlabel('Generation')
    plt.ylabel('Best Time')
    plt.title('Best Time Evolution')
    plt.show()    
if __name__ == "__main__":
    main()

KeyboardInterrupt: 

In [ ]:
def order_crossover(parent1, parent2):
    """
    Order Crossover (OX) for TSP.
    """
    # Ensure the first parent is the shorter one (or equal)
    if len(parent1) > len(parent2):
        parent1, parent2 = parent2, parent1

    # Randomly select a subset of parent1
    start_pos = random.randint(0, len(parent1) - 2)
    end_pos = random.randint(start_pos + 1, len(parent1) - 1)

    child1 = [None] * len(parent1)
    child2 = [None] * len(parent2)

    # Copy the subset to child1 and child2
    child1[start_pos:end_pos] = parent1[start_pos:end_pos]
    child2[start_pos:end_pos] = parent2[start_pos:end_pos]

    # For the rest positions in the child
    # If the position is occupied, insert the first unoccupied element from the other parent
    for i in range(len(parent1)):
        if not child1[i]:
            for gene in parent2:
                if gene not in child1:
                    child1[i] = gene
                    break
        if not child2[i]:
            for gene in parent1:
                if gene not in child2:
                    child2[i] = gene
                    break

    return child1, child2

def crossover(parents, crossover_rate, max_attempts=10000):
    """
    Modified crossover function using Order Crossover (OX) with validity check
    and multiple attempts to generate a valid child.
    """
    if random.random() >= crossover_rate:
        return parents[0][:]
    
    for _ in range(max_attempts):
        child1, child2 = order_crossover(parents[0], parents[1])
        
        if is_valid_individual(child1):
            # Return the child with the better score
            return child1 if get_score(child1) > get_score(child2) else child2
        
    # If after max_attempts we still don't have a valid child, return one of the parents
    return parents[0]


def find_target(nested_list, target):
    # 将二级列表转换为NumPy数组
    nested_array = np.array(nested_list)
    
    # 获取所有子列表的第一个元素
    first_elements = nested_array[:, 0]
    
    # 使用NumPy的索引功能来查找目标数字
    indices = np.where(first_elements == target)[0]
    
    if len(indices) == 0:
        print("Target not found! Nested List:", nested_list, "Target:", target)
    
    return indices

def calculate_enter_time_individual(path):
    individual = path.copy()
    for i in range(len(individual)):
        individual[i] = [[0, 0, 0]] + individual[i]
    timetable = {i: 0 for i in range(1, 8)}
    timetable[7] = float('inf')

    n = len(individual)
    result = [[0] for _ in range(n)]
    variables = {f"tourist_{i + 1}": 1 for i in range(n)}
    while(variables['tourist_1'] <=7 or variables['tourist_2'] <=7 or variables['tourist_3'] <=7):
        time = float("inf")
        ind = 0
        for i in range(len(result)):
            site = variables[f"tourist_{i+1}"]
            if site ==8:
                continue
            current_node = individual[i][site][0]
            previous_node = individual[i][site - 1][0]
            arrive_time = result[i][-1] + adj_matrix[previous_node][current_node]/individual[i][site][2] + individual[i][site-1][1]
            if arrive_time < time:
                ind = i
                time = arrive_time
        print(ind,time)
        site = variables[f"tourist_{ind+1}"]
        arrive_time = time
        index = individual[ind][site][0]
        if index == 3:
            if arrive_time < timetable[index] - 1e-9:
                arrive_time = timetable[index]
                timetable[index] = arrive_time + individual[ind][site][1]
            elif abs(round(arrive_time / 30) * 30 - arrive_time)> 1e-9 and arrive_time > timetable[index] + 1e-9:
                    num = copy.deepcopy(arrive_time)
                    integer_part = int(num)
                    decimal_part = num - integer_part
                    remainder = integer_part % 30
                    difference = 30 - remainder - decimal_part

                    timetable[index] = individual[ind][site][1] + difference + arrive_time
                    arrive_time = difference + arrive_time                                    
            else:
                timetable[index] = timetable[index] +30
        elif index == 7:
            arrive_time = arrive_time
        else:
            if arrive_time > timetable[index]:
                 timetable[index] = arrive_time + individual[ind][site][1]
            else:
                 arrive_time = timetable[index]
                 timetable[index] = timetable[index] + individual[ind][site][1]               
        result[ind].append(arrive_time)
        variables[f"tourist_{ind+1}"] = variables[f"tourist_{ind+1}"] +1       
    return result

def calculate_enter_time_tourist(path):
    result = [0]
    for site in range(1, len(path)):
        current_node = path[site][0]
        previous_node = path[site - 1][0]
        arrive_time = result[-1] + adj_matrix[previous_node][current_node]/path[site][2] + path[site-1][1]
        result.append(arrive_time)
    return result


def is_valid_individual(individual):
    """
    判断个体是否满足约束条件
    """
    enter_time = calculate_enter_time_individual(individual)
    if any(et[-1] > 300 for et in enter_time):
        return False
    if any(et[find_target(ind, 1)[0]] > 240 for ind, et in zip(individual, enter_time)):
        return False
    return True

def is_valid_tourist(tourist):
    """
    判断个体是否满足约束条件
    """
    enter_time = calculate_enter_time_tourist(tourist)
    if enter_time[-1] > 300:
            return False
    if enter_time[find_target(tourist,1)[0]] > 240:
        return False
    return True


def generate_speeds(min_speed,max_speed,average_speed,num,iterations = 10000):
    #使用蒙特卡洛方法生成
    best_numbers = None
    best_average = 0
    
    for _ in range(iterations):
        # 生成一个包含 num 个在 min_speed 到 max_speed 之间的随机浮点数的列表
        numbers = [random.uniform(min_speed, max_speed) for _ in range(num)]
        
        # 计算生成数字的平均值
        average = sum(numbers) / len(numbers)
        
        # 检查平均值是否在所需范围内
        if average <= average_speed and average > best_average:
            best_numbers = numbers
            best_average = average
    best_numbers = [speed*1000/60 for speed in best_numbers]
    if best_numbers:
        return(best_numbers)
    else:
        print("在约束条件内未找到有效的数字组合。")

def mutate(individual, mutation_rate, max_attempts=10000):
    mutated_individual = copy.deepcopy(individual)
    global range_dict
    for _ in range(max_attempts):
        for ind in range(len(individual)):
            for i in range(0, len(mutated_individual[ind])-1):
                if random.random() < mutation_rate:
                    dimension = mutated_individual[ind][i][0]
                    if isinstance(range_dict[dimension], int):
                        mutated_individual[ind][i][1] = range_dict[dimension]
                    else:
                        mutated_individual[ind][i][1] = random.randint(range_dict[dimension][0], range_dict[dimension][1])
            if random.random() < mutation_rate:
                index1 = random.randint(0, len(mutated_individual[ind]) - 2)
                index2 = random.randint(0, len(mutated_individual[ind]) - 2)
                while index2 == index1:
                    index2 = random.randint(0, len(mutated_individual[ind]) - 2)
                    mutated_individual[ind][index1], mutated_individual[ind][index2] = mutated_individual[ind][index2], mutated_individual[ind][index1]
        if is_valid_individual(mutated_individual):
            break
    else:
        # 如果达到了最大尝试次数，可以返回原始的个体或采取其他措施
        mutated_individual = individual
    return mutated_individual

def get_score(individual):
    """
    评价一个个体
    """
    sightsee = 0
    entertime = calculate_enter_time_individual(individual)
    for i in range(len(entertime)):
        sightsee = sightsee + (330 - entertime[i][-1])
    for ind in range(len(individual)):
        for i in range(len(individual[ind])-1):
            sightsee = sightsee + individual[ind][i][1]
    return sightsee

def generate_individual(tourist_size):
    global range_dict
    keys = list(range_dict.keys())
    individual = []
    while len(individual) < tourist_size:
        path = [
            [dimension, range_dict[dimension] if isinstance(range_dict[dimension], int) else
             random.randint(range_dict[dimension][0], range_dict[dimension][1])]
            for dimension in keys[:6]
        ]
        
        # 随机打乱 path 中的元素顺序
        random.shuffle(path)
        path.append([7, float('inf')])
        # 添加速度信息
        speeds = [2000/60, 2000/60, 2000/60, 2000/60, 2000/60, 2000/60, 2000/60]
        for i in range(len(path)):
            path[i].append(speeds[i])
        if is_valid_tourist(path):
            individual.append(path)
    return individual

def initialize_population(population_size,tourist_size):
    """
    初始化种群，随机生成不同的路径作为初始解
    :param population_size: 种群大小
    :return: 生成的种群
    """
    # 构建一个字典表示对应的游览时间
    
    population = []
    while len(population) < population_size:
        individual = generate_individual(tourist_size)
        if is_valid_individual(individual):
            population.append(individual)
    return population

def elite_preserve(population, elite_size):
    """
    精英保留函数，从种群中选择最优个体作为精英个体，将精英个体直接传递到下一代
    :param population: 种群
    :param elite_size: 精英个体数量
    :return: 精英个体列表
    """
    elites = sorted(population, key=lambda path: get_score(path), reverse=True)[:elite_size]
    return elites

def select_parents(population):
    """
    从种群中选择两个父代
    """ 
    parents = random.sample(population, 2)
    parents_copy = copy.deepcopy(parents)  # 创建父代的深拷贝
    return parents_copy

def tournament_selection(population, tournament_size):
    """
    锦标赛选择，从种群中选择父代个体
    :param population: 种群
    :param tournament_size: 锦标赛大小
    :return: 选择出的父代个体
    """
    competitors = random.sample(population, tournament_size)
    winner = max(competitors, key=lambda individual: get_score(individual))
    return winner

def factor_probability4(population,parents, Pmin, Pmax, A = 9.903438):
    scores = [get_score(path) for path in population]  # 计算所有路径的分数
    fmean = np.mean(scores)
    f_ = max(get_score(parents[0]),get_score(parents[1]))
    if f_ <= fmean:
        return (Pmax - Pmin)/(1 + np.exp(A * 2 * (fmean - f_) / (fmean - Pmin -1))) + Pmin
    else:
        return Pmax

# 为给定的个体生成一个有效的邻居
def get_valid_neighbor(a):
    new_a = copy.deepcopy(a)  # 创建a的深拷贝，确保原始列表不被修改
    while True:
        sublist_idx = np.random.randint(len(new_a))  # 随机选择一个子列表
        element_idx = np.random.choice([i for i, x in enumerate(new_a[sublist_idx]) if x[0] != 7])  # 选择一个非7的元素索引
        idx_val = new_a[sublist_idx][element_idx][0]
        range_val = range_dict[idx_val]
        # 根据range_val更改元素的值
        if isinstance(range_val, tuple):
            new_a[sublist_idx][element_idx][1] = np.random.randint(range_val[0], range_val[1]+1)
        else:
            new_a[sublist_idx][element_idx][1] = range_val
        # 检查新的个体是否有效
        if is_valid_individual(new_a):
            break
    return new_a


# 对个体应用模拟退火算法
def simulated_annealing_for_individual(initial_temperature, alpha, num_iterations, individual):
    current_individual = copy.deepcopy(individual)  # 创建individual的深拷贝
    current_score = get_score(current_individual)  # 计算当前个体的得分
    
    T = initial_temperature  # 设置初始温度
    for _ in range(num_iterations):
        neighbor_individual = get_valid_neighbor(current_individual)  # 获取邻居
        neighbor_score = get_score(neighbor_individual)  # 计算邻居的得分

        delta_E = neighbor_score - current_score  # 计算得分差
        # 如果新得分更好或满足某个条件，接受新的个体
        if delta_E > 0 or np.random.rand() < math.exp(delta_E / T):
            current_individual = neighbor_individual
            current_score = neighbor_score

        T *= alpha  # 降低温度
    return current_individual

def save_population(population, filename='population.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(population, f)

def load_population(filename='population.pkl'):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)
    else:
        return None

def ant_colony_search_for_initial_population(num_solutions_required):
    solutions = []
    best_score = float('-inf') # 初始化best_score
    best_solution = None      # 初始化best_solution
    while len(solutions) < num_solutions_required:
        for iteration in range(iterations):
            for ant in range(num_ants):
                new_solution = copy.deepcopy(a)
                
                for sublist in new_solution:
                    for element in sublist:
                        if element[0] != 7:
                            element[1] = choose_value(pheromone_matrix, element[0])
                
                # 确保解的有效性
                if is_valid_individual(new_solution):
                    solutions.append(new_solution)
                    if len(solutions) >= num_solutions_required:
                        break
                
                # 如果得到的解是最佳解，则更新信息素
                score = get_score(new_solution)
                if score > best_score:
                    best_score = score
                    best_solution = new_solution
                    for key in pheromone_matrix.keys():
                        pheromone_matrix[key] += pheromone_deposit
                
                # 信息素蒸发
                for key in pheromone_matrix.keys():
                    pheromone_matrix[key] *= (1 - pheromone_evaporation)
    return solutions


import random
import copy
import numpy as np
import matplotlib.pyplot as plt
import math
def main():
    tourist_size = 3 #旅行团数量

    population_size = 100 # 种群大小
    num_generations = 1000 # 迭代次数
    tournament_size = 5 #锦标赛大小
    elite_size = 3 # 精英保留个体数量
    mutation_size = population_size - elite_size - tournament_size
    initial_temperature = 500  # 初始温度
    alpha = 0.98              # 温度降低率
    sa_iterations = 5  # 设置每个个体的模拟退火迭代次数

    global range_dict
    range_dict = {
        1: (10, 30),
        2: (20, 60),
        3: 30,
        4: (30, 60),
        5: (20, 60),
        6: (30, 60),
    }

    global adj_matrix
    adj_matrix = (
    (  0, 300, 360, 210, 530, 475, 500, 690),
    (300,   0, 380, 270, 230, 285, 200, 390),
    (360, 380,   0, 510, 230, 665, 580, 770),
    (210, 270, 510,   0, 470, 265, 450, 640),
    (530, 230, 230, 470,   0, 515, 360, 550),
    (475, 285, 665, 265, 515,   0, 460, 650),
    (500, 200, 580, 450, 360, 460,   0, 190),
    (690, 390, 770, 640, 550, 650, 190,   0))
   # 初始化种群
    loaded_population = load_population()
    if loaded_population:
        population = loaded_population
    else:
        population = ant_colony_search_for_initial_population(population_size)  
    # 记录每代的最佳时间
    best_times = []
        # 迭代指定的代数
    for generation in range(num_generations):
        new_population = []
        elites = elite_preserve(population, elite_size)       
        new_population.extend(elites)  # 将精英个体添加到新种群
        for _i1 in range(mutation_size):
            # 选择父代
            parents = select_parents(population)
            # 通过交叉产生子代
            crossover_rate = 1 - factor_probability4(population,parents, Pmin = 0.3, Pmax = 0.75, A = 9.903438)
            child = crossover(parents,crossover_rate)
            # 对子代进行变异
            mutation_rate = 1 - factor_probability4(population,parents, Pmin = 0.01, Pmax = 0.075, A = 9.903438)
            child = mutate(child, mutation_rate)
            # 将子代添加到新种群
            child = simulated_annealing_for_individual(initial_temperature, alpha, sa_iterations, child)  # 对个体应用模拟退火
            new_population.append(child)
            
        for _i2 in range(tournament_size):
            child = tournament_selection(population, tournament_size)
            new_population.append(child) 
        # 找到当前代的最佳路径和
        current_best_path = max(population, key=lambda path: get_score(path))
        current_best_distance = get_score(current_best_path)
        save_population(population)        
        # 记录最佳时间
        best_times.append(current_best_distance)

        # 打印当前代的最佳路径和距离
        print(f"Generation {generation + 1}:")
        print("最佳路径：", current_best_path)
        print("最佳时间：", current_best_distance)
        print()
        # 更新种群
        population = new_population

    # 绘制迭代图
    plt.plot(range(1, num_generations + 1), best_times)
    plt.xlabel('Generation')
    plt.ylabel('Best Time')
    plt.title('Best Time Evolution')
    plt.show()    
if __name__ == "__main__":
    main()

NameError: name 'os' is not defined

In [ ]:
a = [[[6, 57, 33.333333333333336], [1, 18, 33.333333333333336], [2, 34, 33.333333333333336], [4, 47, 33.333333333333336], [3, 30, 33.333333333333336], [5, 32, 33.333333333333336], [7, inf, 33.333333333333336]], [[2, 50, 33.333333333333336], [4, 37, 33.333333333333336], [3, 30, 33.333333333333336], [5, 45, 33.333333333333336], [1, 30, 33.333333333333336], [6, 30, 33.333333333333336], [7, inf, 33.333333333333336]], [[5, 33, 33.333333333333336], [3, 30, 33.333333333333336], [4, 41, 33.333333333333336], [2, 21, 33.333333333333336], [1, 12, 33.333333333333336], [6, 39, 33.333333333333336], [7, inf, 33.333333333333336]]]

def get_valid_neighbor(a):
    new_a = copy.deepcopy(a)
    while True:
        # Select a random sublist
        sublist_idx = np.random.randint(len(new_a))
        # Select a random element in the sublist that doesn't have an index of 7
        element_idx = np.random.choice([i for i, x in enumerate(new_a[sublist_idx]) if x[0] != 7])
        # Change its second value within its range
        idx_val = new_a[sublist_idx][element_idx][0]
        range_val = range_dict[idx_val]
        if isinstance(range_val, tuple):
            new_a[sublist_idx][element_idx][1] = np.random.randint(range_val[0], range_val[1]+1)
        else:
            new_a[sublist_idx][element_idx][1] = range_val
        # Check if the new individual is valid
        if is_valid_individual(new_a):
            break
    return new_a

def simulated_annealing_maximize_v2(initial_temperature, alpha, num_iterations, a):
    current_a = copy.deepcopy(a)
    current_score = get_score(current_a)
    best_a = copy.deepcopy(current_a)
    best_score = current_score
    
    T = initial_temperature
    for i in range(num_iterations):
        neighbor_a = get_valid_neighbor(current_a)
        neighbor_score = get_score(neighbor_a)
        
        # Calculate delta E
        delta_E = neighbor_score - current_score
        if delta_E > 0 or np.random.rand() < math.exp(delta_E / T):
            current_a = copy.deepcopy(neighbor_a)
            current_score = neighbor_score
            if current_score > best_score:
                best_score = current_score
                best_a = copy.deepcopy(current_a)
        
        # Reduce the temperature
        T *= alpha

    return best_a, best_score
print(get_score(a))
for i in range(1000):
    a ,_ = simulated_annealing_maximize_v2(500, 0.98, 5, a)
    print(get_score(a))


743.1999999999999
743.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
757.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
760.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.1999999999999
763.199999

KeyboardInterrupt: 

In [ ]:
def factor_probability4(population,parents, Pmin, Pmax, A = 9.903438):
    scores = [get_score(path) for path in population]  # 计算所有路径的分数
    fmean = np.mean(scores)
    f_ = max(get_score(parents[0]),get_score(parents[1]))
    if f_ <= fmean:
        return (Pmax - Pmin)/(1 + np.exp(A * 2 * (fmean - f_) / (fmean - Pmin -1))) + Pmin
    else:
        return Pmax

In [4]:
def calculate_enter_time_individual(path):
    individual = path.copy()
    for i in range(len(individual)):
        individual[i] = [[0, 0, 0]] + individual[i]
    timetable = {i: 0 for i in range(1, 8)}
    timetable[7] = float('inf')

    n = len(individual)
    result = [[0] for _ in range(n)]
    variables = {f"tourist_{i + 1}": 1 for i in range(n)}
    while(variables['tourist_1'] <=7 or variables['tourist_2'] <=7 or variables['tourist_3'] <=7):
        time = float("inf")
        ind = 0
        for i in range(len(result)):
            site = variables[f"tourist_{i+1}"]
            if site ==8:
                continue
            current_node = individual[i][site][0]
            previous_node = individual[i][site - 1][0]
            arrive_time = result[i][-1] + adj_matrix[previous_node][current_node]/individual[i][site][2] + individual[i][site-1][1]
            if arrive_time < time:
                ind = i
                time = arrive_time
        site = variables[f"tourist_{ind+1}"]
        arrive_time = time
        index = individual[ind][site][0]
        print(time,ind)
        if index == 3:
            if arrive_time < timetable[index] - 1e-9:
                arrive_time = timetable[index]
                timetable[index] = arrive_time + individual[ind][site][1]
            elif abs(round(arrive_time / 30) * 30 - arrive_time)> 1e-9 and arrive_time > timetable[index] + 1e-9:
                    num = copy.deepcopy(arrive_time)
                    integer_part = int(num)
                    decimal_part = num - integer_part
                    remainder = integer_part % 30
                    difference = 30 - remainder - decimal_part

                    timetable[index] = individual[ind][site][1] + difference + arrive_time
                    arrive_time = difference + arrive_time                                    
            else:
                timetable[index] = timetable[index] +30
        elif index == 7:
            arrive_time = arrive_time
        else:
            if arrive_time > timetable[index]:
                 timetable[index] = arrive_time + individual[ind][site][1]
            else:
                 arrive_time = timetable[index]
                 timetable[index] = timetable[index] + individual[ind][site][1]               
        result[ind].append(arrive_time)
        variables[f"tourist_{ind+1}"] = variables[f"tourist_{ind+1}"] +1       
    return result

path = [
    [[6, 40, 33.333333333333336], [1, 18, 33.333333333333336], [2, 60, 33.333333333333336], 
     [4, 37, 33.333333333333336], [3, 30, 33.333333333333336], [5, 21, 33.333333333333336], [7, float('inf'), 33.333333333333336]],

    [[2, 25, 33.333333333333336], [4, 32, 33.333333333333336], [3, 30, 33.333333333333336], 
     [5, 26, 33.333333333333336], [1, 10, 33.333333333333336], [6, 52, 33.333333333333336], [7, float('inf'), 33.333333333333336]],

    [[5, 33, 33.333333333333336], [3, 30, 33.333333333333336], [4, 53, 33.333333333333336], 
     [2, 34, 33.333333333333336], [1, 25, 33.333333333333336], [6, 32, 33.333333333333336], [7, float('inf'), 33.333333333333336]]
]
calculate_enter_time_individual(path)

10.799999999999999 1
14.249999999999998 2
14.999999999999998 0
42.7 1
55.199999999999996 2
61.0 0
88.80000000000001 1
90.4 0
104.1 2
127.95 1
157.3 0
162.5 1
164.0 2
178.5 1
208.4 0
209.4 2
236.2 1
240.4 2
247.95 0
278.1 2
288.45 0


[[0, 14.999999999999998, 61.0, 90.4, 157.3, 210.0, 247.95, 288.45],
 [0, 10.799999999999999, 42.7, 90.0, 127.95, 162.5, 178.5, 236.2],
 [0, 14.249999999999998, 60.0, 104.1, 164.0, 209.4, 240.4, 278.1]]